In [ ]:
import os
import pandas as pd
from utils import clean_data

rt = './data'
csv_file_paths = os.listdir(rt)
for csv_path in csv_file_paths:
    df = pd.read_csv(f'{rt}/{csv_path}')
    df = clean_data(df) # Utilisation d'un heler générique de nettoyage

/home/mathieu/prj/olist-learning/utils.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/utils.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col],
/home/mathieu/prj/olist-learning/utils.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
 

Les données sont désormais nettoyées